In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import json
from time import sleep, perf_counter as pf
import re

In [13]:
def raze_list(l):
    out = []
    for x in l:
        if type(x) == list:
            out += raze_list(x)
        else:
            out.append(x)     
    return out

In [3]:
def post_link(tag):
    if tag.name == 'a':
        if tag.has_attr('data-tn-element') and tag.has_attr('href'):
            return 'jobtitle' in tag['data-tn-element'].lower().replace(' ', '')
    return False

def is_next_link(tag):
    return (not post_link(tag)) and tag.name == 'a' and 'next' == tag.text.lower()[:4] and tag.has_attr('href')

Here we search for job postings on [Indeed](https://www.indeed.ca/) matching "data science" in Ontario.

In [1]:
url = 'https://www.indeed.ca/'

In [4]:
next_link = 'jobs?q=data+science&l=ON'
i = 0
s = []
with requests.Session() as sess:
    while next_link is not None and i < 50:
        next_link = next_link.strip('/')
        if next_link[:22] != url:
            next_link = url + next_link
        print(next_link)
        tmp = BeautifulSoup(sess.get(next_link).text, 'html.parser')
        t1 = pf()
        s += [url + x['href'].strip('/') for x in tmp.find_all(post_link)]# if x.has_attr('href')]
        next_link = tmp.find('div', attrs = {'class':'pagination'}).find(is_next_link)
        if next_link is not None:
            next_link = next_link['href']
        i += 1
        sleep(max(2 - pf() + t1, 0))

https://www.indeed.ca/jobs?q=data+science&l=ON
https://www.indeed.ca/jobs?q=data+science&l=ON&start=20
https://www.indeed.ca/jobs?q=data+science&l=ON&start=40
https://www.indeed.ca/jobs?q=data+science&l=ON&start=60
https://www.indeed.ca/jobs?q=data+science&l=ON&start=80
https://www.indeed.ca/jobs?q=data+science&l=ON&start=100
https://www.indeed.ca/jobs?q=data+science&l=ON&start=120
https://www.indeed.ca/jobs?q=data+science&l=ON&start=140
https://www.indeed.ca/jobs?q=data+science&l=ON&start=160
https://www.indeed.ca/jobs?q=data+science&l=ON&start=180
https://www.indeed.ca/jobs?q=data+science&l=ON&start=200
https://www.indeed.ca/jobs?q=data+science&l=ON&start=220
https://www.indeed.ca/jobs?q=data+science&l=ON&start=240
https://www.indeed.ca/jobs?q=data+science&l=ON&start=260
https://www.indeed.ca/jobs?q=data+science&l=ON&start=280
https://www.indeed.ca/jobs?q=data+science&l=ON&start=300
https://www.indeed.ca/jobs?q=data+science&l=ON&start=320
https://www.indeed.ca/jobs?q=data+science&l=O

Saving progress.

In [5]:
with open('post_links.json', 'w') as f:
    json.dump(s, f)

In [7]:
with open('post_links.json', 'r') as f:
    s = json.load(f)

In [6]:
def employer(tag):
    return tag.has_attr('class') and max(['companyreviewheading' in x.lower().replace('-', '') for x in tag['class']])
def title(tag):
    return tag.has_attr('class') and max(['jobinfoheadertitle' in x.lower().replace('-', '') for x in tag['class']])
def props(tag):
    return 'div' == tag.name and tag.has_attr('class') and max(['jobmetadataheaderitem' in x.lower().replace('-', '') for x in tag['class']])
def desc(tag):
    return 'div' == tag.name and tag.has_attr('class') and max(['jobcomponentdescription' in x.lower().replace('-', '') for x in tag['class']])
def desc_text(tag):
    return 'div' == tag.name and 0 == len(tag.attrs)
def desc_list(tag):
    return 'p' == tag.name or 'li' == tag.name

Getting postings.

In [7]:
posts = []
with requests.Session() as sess:
    try:
        for link in s:
            #print(link)
            tmp = BeautifulSoup(sess.get(link).text, 'html.parser')
            info = dict()
            info['link'] = link
            x = tmp.find(employer)
            if x is not None:
                info['employer'] = x.text
            if tmp.find(title) is not None:
                info['title'] = tmp.find(title).text
            else:
                info['title'] = ''
            info['props'] = [x.text for x in tmp.find_all(props)]
            for script in tmp(["script", "style"]):
                script.decompose()  
            info['text'] = tmp.get_text()
            info['exp'] = [x for x in info['text'] if 'year' in x and ('experience' in x or 'Experience' in x)]
            posts.append(info)
    except Exception as e:
        print(link)
        raise e

Saving progress.

In [8]:
with open('indeed_posts.json', 'w') as f:
    json.dump(posts, f)

In [2]:
with open('indeed_posts.json', 'r') as f:
    posts = json.load(f)

The number of postings analysed:

In [3]:
len(posts)

1250

A DataFrame containing the job postings.

In [4]:
data = pd.read_json(json.dumps(posts), orient = 'records')
data.head()

,employer,exp,link,props,text,title
0,Sun Life Financial,[],https://www.indeed.ca/pagead/clk?mo=r&ad=-6NYl...,[],"\n\nData Scientist - Waterloo, ON - Indeed.com...",Data Scientist
1,Rakuten Kobo Inc.,[],https://www.indeed.ca/pagead/clk?mo=r&ad=-6NYl...,[],"\n\nSoftware Developer, Big Data - Toronto, ON...","Software Developer, Big Data"
2,Intact,[],https://www.indeed.ca/pagead/clk?mo=r&ad=-6NYl...,[],"\n\nData Scientist - Big Data - Toronto, ON - ...",Data Scientist - Big Data
3,AT&T,[],https://www.indeed.ca/pagead/clk?mo=r&ad=-6NYl...,[],"\n\nTechnical Specialist, Network Design - Dat...","Technical Specialist, Network Design - Data Ce..."
4,Hydro Ottawa,[],https://www.indeed.ca/rc/clk?jk=9a887b02b03bfa...,"[$77,188 a year]","\n\nManager, Data Science - Ottawa, ON - Indee...","Manager, Data Science"


Getting a list of Departments and Agencies in the Federal Public Service.

In [5]:
others = '''Government of Canada
Canadian Security Intelligence Service
National Capital Commission
Canada Investment and Savings
Canadian Forces Non-Public Funds
Royal Canadian Mint
Canadian Forces Morale and Welfare Services
Bank of Canada
Canadian Air Transport Security Authority
Security Intelligence Review Committee'''.split("\n")
others = [x.lower().strip().replace(" ", '') for x in others]
others

agencies = pd.read_html('https://www.canada.ca/en/treasury-board-secretariat/services/innovation/human-resources-statistics/population-federal-public-service-department.html')
agencies = agencies[0]['Departments and Agencies']['2000'].str.lower().str.replace('[^a-z]', '').str.strip().tolist() + others

Separating out the federal public sector.  (Note that the public sector at other levels of governement have been grouped with the private sector.)  Here we display the first five rows of the federal public service data.

In [6]:
gov = data[data['employer'].fillna('').str.lower().str.replace('[^a-z]', '').str.strip().isin(agencies)]#.head()
priv = data[~data['employer'].fillna('').str.lower().str.replace('[^a-z]', '').str.strip().isin(agencies)]#.head()
gov.head()

,employer,exp,link,props,text,title
7,Government of Canada,[],https://www.indeed.ca/rc/clk?jk=7ff10eb9b80c01...,"[$108,528 - $141,426 a year]","\n\nDirector, Data Analytics - Ottawa, ON - In...","Director, Data Analytics"
15,Canadian Security Intelligence Service,[],https://www.indeed.ca/rc/clk?jk=3022e9338495f0...,"[$89,400 - $108,820 a year, Permanent]","\n\nData Scientist - Ottawa, ON - Indeed.ca\n\...",Data Scientist
17,Government of Canada,[],https://www.indeed.ca/rc/clk?jk=4ddb6d46a67895...,"[$70,439 - $86,213 a year]","\n\nData Base Analyst - Ottawa, ON - Indeed.co...",Data Base Analyst
20,Government of Canada,[],https://www.indeed.ca/rc/clk?jk=8aa6d0ab99926b...,"[$57,426 - $79,234 a year]","\n\nAnalyst - Ottawa, ON - Indeed.com\n\n\nFin...",Analyst
32,Government of Canada,[],https://www.indeed.ca/rc/clk?jk=dd3fad433a4f35...,"[$35,267 - $70,309 a year]",\n\nPhysical Scientist / Physical science poli...,Physical Scientist / Physical science policy a...


The number of private sector job postings.

In [7]:
len(priv)

1210

The number of public sector job postings.

In [8]:
len(gov)

40

A unique set of private sector employers.

In [9]:
set(priv.employer)

{'ADGA Group',
 'AECOM',
 'AGAT Laboratories',
 'AT&T',
 'AbbVie',
 'Abbott Laboratories',
 'Academy for Mathematics & English',
 'Accenture',
 'Accreditation Canada',
 'Adastra Corporation',
 'Adventure Place',
 'Affinity Systems',
 'Aimia',
 'Ainsworth',
 'Air Canada Vacations',
 'Algonquin College',
 'AllianceData',
 'Allianz Global Assistance',
 'Alterna Savings',
 'Alzheimer Society Peel',
 'Amazon.com',
 'Aon Canada',
 'Aphria Inc.',
 'Apple',
 'Applied Systems Inc.',
 'Applyboard',
 'AstraZeneca',
 'Aurora Cannabis, Inc',
 'Ausenco',
 'Autodesk',
 'Avanade',
 'Aviva',
 'Axiom Real-Time Metrics',
 'BBM',
 'BGIS',
 'BMO Financial Group',
 'Baird & Associates',
 'Barrick Gold Corporation',
 'Baycrest',
 'Bayer Canada',
 "Beau's All Natural Brewing Co.",
 "Beau's All Natural Brewing Company",
 'BelairDirect',
 'Bell',
 'BiblioCommons',
 'BlackBerry',
 'Bodycote',
 'BrainStation',
 'Brock University',
 'Brookfield Properties',
 'CAPCO',
 'CENX',
 'CH2M',
 'CHRC',
 'CI Investments Inc

A unique set of public sector employers with postings on Indeed.

In [10]:
set(gov.employer)

{'Canadian Air Transport Security Authority',
 'Canadian Forces Morale and Welfare Services',
 'Canadian Security Intelligence Service',
 'Communications Security Establishment',
 'Government of Canada',
 'National Research Council Canada',
 'Royal Canadian Mint'}

In [11]:
def compound_list(l):
    d = ''
    for string in l:
        d = d + string 
    return d 

In [14]:
gov_desc = compound_list(raze_list(gov['text'])).lower()
priv_desc = compound_list(raze_list(priv['text'])).lower()

The list of languages and technologies mentioned in these job postings alongside their frequencies, separated into private and public sector.

In [15]:
lang = {'python':re.compile('python'), 'r': re.compile('\\br\\b'), 'sql':re.compile('sql'),\
      'sas':re.compile('\\bsas\\b|statistical analysis system'), 'excel': re.compile('\\bexcel\\b'), \
       'spss':re.compile('spss|statistical package for (the ){0,1}social science'), \
      'hadoop':re.compile('hadoop'), 'kibana':re.compile('kibana'), 'tableau':re.compile('tableau'), \
      'pig':re.compile('\\bpig\\b'), 'stata':re.compile('stata'), 'powerbi':re.compile('power bi'), \
      'java':re.compile('java'), 'c/c++':re.compile('\\bc(\\+\\+){0,1}\\b'), 'hive':re.compile('\\bhive\\b'), \
      'matlab':re.compile('matlab'), 'ruby':re.compile('ruby'), 'perl':re.compile('\\bperl\\b'), 'hbase':re.compile('hbase'), \
      'spark':re.compile('spark'), 'php':re.compile('\\bphp\\b'), 'scala':re.compile('\\bscala\\b')}
gov_results = dict()
priv_results = dict()
for key in lang:
    priv_results[key] = len(re.findall(lang[key], priv_desc))
    gov_results[key] = len(re.findall(lang[key], gov_desc))

Private sector results:

In [16]:
priv_results

{'python': 484,
 'r': 461,
 'sql': 860,
 'sas': 157,
 'excel': 248,
 'spss': 33,
 'hadoop': 473,
 'kibana': 3,
 'tableau': 108,
 'pig': 17,
 'stata': 10,
 'powerbi': 39,
 'java': 409,
 'c/c++': 457,
 'hive': 261,
 'matlab': 39,
 'ruby': 66,
 'perl': 19,
 'hbase': 175,
 'spark': 230,
 'php': 28,
 'scala': 111}

Public sector results:

In [17]:
gov_results

{'python': 7,
 'r': 5,
 'sql': 9,
 'sas': 8,
 'excel': 8,
 'spss': 2,
 'hadoop': 0,
 'kibana': 0,
 'tableau': 0,
 'pig': 0,
 'stata': 1,
 'powerbi': 1,
 'java': 12,
 'c/c++': 13,
 'hive': 0,
 'matlab': 1,
 'ruby': 0,
 'perl': 1,
 'hbase': 0,
 'spark': 1,
 'php': 2,
 'scala': 0}

Saving to .csv.

In [23]:
gov_lang = [(key,gov_results[key]) for key in gov_results]
gov_lang = pd.DataFrame(gov_lang, columns = ['lang', 'freq'])
gov_lang.to_csv('gov_lang.csv', index=False)

In [24]:
priv_lang = [(key,priv_results[key]) for key in priv_results]
priv_lang = pd.DataFrame(priv_lang, columns = ['lang', 'freq'])
priv_lang.to_csv('priv_lang.csv', index=False)

Searching for keywords in non-salary compensations and perks mentioned in the job postings, separated out by public and private sector.

In [18]:
pros = {'opportunity':re.compile('opportunity'), 'growth':re.compile('growth'), \
               'advancement':re.compile('advancement'), 'bonus':re.compile('bonus'), \
               'balance':re.compile('balance'), 'benefits':re.compile('benefits'), \
               'travel':re.compile('travel'), 'flexibility':re.compile('flexibility|flexible'), \
       'diversity':re.compile('diverse|diversity')}

gov_pros = dict()
priv_pros = dict()
for key in pros:
    priv_pros[key] = len(re.findall(pros[key], priv_desc))
    gov_pros[key] = len(re.findall(pros[key], gov_desc))
    

Public sector:

In [19]:
gov_pros

{'opportunity': 26,
 'growth': 4,
 'advancement': 0,
 'bonus': 1,
 'balance': 4,
 'benefits': 23,
 'travel': 35,
 'flexibility': 20,
 'diversity': 61}

Private sector:

In [20]:
priv_pros

{'opportunity': 885,
 'growth': 415,
 'advancement': 41,
 'bonus': 66,
 'balance': 127,
 'benefits': 465,
 'travel': 314,
 'flexibility': 514,
 'diversity': 716}

Saving to .csv.

In [28]:
gov_pros = [(key,gov_pros[key]) for key in gov_pros]
gov_pros = pd.DataFrame(gov_pros, columns = ['pro', 'freq'])
gov_pros.to_csv('gov_pros.csv', index=False)

In [29]:
priv_pros = [(key,priv_pros[key]) for key in priv_pros]
priv_pros = pd.DataFrame(priv_pros, columns = ['pro', 'freq'])
priv_pros.to_csv('priv_pros.csv', index=False)